In [1]:
import os
import cv2
import numpy as np
from moviepy.editor import VideoFileClip
import datetime
import time
from tqdm import notebook
import re

In [2]:
# [ 참고 사항 ]
# mp4 파일 : 5개 사용 
    # 2분 초과 데이터 : 1개 포함
    # 초반부터 2명 이상의 얼굴이 나오는 데이터 : 2개 포함
    # -> 우리는 1명의 앵커 얼굴만 나오는 데이터가 필요함

In [3]:
# 코드 실행시간 측정
# start = time.time()

In [4]:
# Whole_MOV_List 함수 : 전체 동영상 리스트 받아오기
# mp4 파일이 들어있는 폴더명 입력
def Whole_MOV_List(MOV_Directory):
    MOV_List = os.listdir(MOV_Directory)

    # ./MOV/파일이름.mp4 형태로 파일명 변형
    Using_MOV = np.array([MOV_Directory+'/'+i for i in MOV_List])
    
    return Using_MOV

In [5]:
# 전체 파일 숫자 : 5개
# a = Whole_MOV_List('./MOV')
# print(a)

In [6]:
# Two_min_under_MOV 함수 : 2분 이하 동영상 선별 

def Two_min_under_MOV(MOV_List):
    
    # Using_List : 사용할 영상들 file_name 모음 
    Using_List = MOV_List[:]
    
    # del_index : 삭제할 영상들 index 모음
    del_index = np.array([])
    del_index = del_index.astype(int)
    
    for i in notebook.tqdm(range(len(MOV_List))):
        clip = VideoFileClip(MOV_List[i])
        play_time = clip.duration
        
        # 재생 시간이 2분을 넘는 경우 -> Using_list에서 제외
        if play_time > 60*2 :
            print('Except MOV_file name :', Using_List[i])
            del_index = np.append(del_index, [i])
            
    Using_List = np.delete(Using_List, del_index)
            
    # 2분 미만인 동영상 리스트
    return Using_List

In [7]:
# 남은 파일 숫자 ; 4개
# b = Two_min_under_MOV(a)
# print('\n', b)

In [8]:
# 전처리 1-1 : 앵커 1명의 얼굴로 시작되는 영상 리스트 찾기

def one_anchor_starts_MOV(MOV_List):
     
    Using_MOV = MOV_List[:]
    
    # 파일 확장자를 제외한 순수한 파일명
    Origin_file_name = np.array([])
    Origin_file_name = Origin_file_name.astype(str)
    
    # 가중치 파일 경로
    cascade_filename = 'haarcascade_frontalface_alt.xml'
    # 모델 불러오기
    cascade = cv2.CascadeClassifier(cascade_filename)
    
    # 첫 장면에서 인식한 얼굴의 수
    face_num_list = np.array([])
    face_num_list = face_num_list.astype(int)
    
    for i in notebook.tqdm(range(len(MOV_List))):
        video = cv2.VideoCapture(MOV_List[i]) #video name change
        ret, frame = video.read()
        
        # 첫 번째로 등장하는 프레임만 취급
        frame[0]

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        results = cascade.detectMultiScale(gray,
                                          scaleFactor=2,
                                          minNeighbors=3,
                                          minSize=(20,20))
        
        face_num_list = np.append(face_num_list, [len(results)])
    
    # del_index : 삭제할 영상들 index 모음
    del_index = np.array([])
    del_index = del_index.astype(int)
    
    # 앵커 1명으로 시작하는 영상 찾기
    for k in range(len(face_num_list)):
        if face_num_list[k]!=1:
            del_index = np.append(del_index, [k])
    
    Using_MOV = np.delete(Using_MOV, del_index)
    
    for MOV in Using_MOV:
        MOV_origin = re.sub(pattern='.\/MOV\/', repl='', string=MOV)
        MOV_origin = re.sub(pattern='.mp4', repl='', string=MOV_origin)
        
        Origin_file_name = np.append(Origin_file_name, MOV_origin)

    # 파일 처리 쉽게 할 수 있는 것을 목표 
    # -> MOV 원본 파일 주소, MOV 파일확장자 지운 버전 모두 제공 
    return Using_MOV, Origin_file_name

In [9]:
# 남은 파일 숫자 : 2개
# Using_MOV, Origin_file_name = one_anchor_starts_MOV(b)
# print(Using_MOV, '\n')
# print(Origin_file_name)

In [10]:
# 실행시간 결과
# sec = time.time()-start
# times = str(datetime.timedelta(seconds=sec)).split(".")
# times = times[0]
# print(times)